# Домашнее задание 3

## Общая информация

Дата выдачи: 15.03.2016

Срок сдачи: 25.03.2016 23:59MSK

### О задании
Домашнее задание №3 направлено на реализацию наивного байесвского классификатора для многоклассового случая.

### Оценивание и штрафы
Каждая из задач имеет определенную «стоимость» (указана в скобках около задачи). Максимально допустимая оценка за работу — 10 баллов. Обратите внимание, что только за реализацию функций без подтверждения их корректной работы оценка выставляться не будет.

Сдавать задание после указанного срока сдачи нельзя. При выставлении неполного балла за задание в связи с наличием ошибок на усмотрение проверяющего предусмотрена возможность исправить задание на указанных в ответном письме условиях.

Задание выполняется самостоятельно. «Похожие» решения считаются плагиатом и все задействованные студенты (в том числе те, у кого списали) не могут получить за него больше 0 баллов. Если вы нашли решение какого-то из заданий (или его часть) в открытом источнике, необходимо указать ссылку на этот источник в отдельном блоке в конце данного ноутбука (скорее всего вы будете не единственным, кто это нашел, поэтому чтобы исключить подозрение в плагиате, необходима ссылка на источник). 

### Формат сдачи
Для сдачи задания скачайте *(File -> Download as -> IPython notebook)* и переименуйте получившийся файл \\*.ipynb в соответствии со следующим форматом: *Username_(group)_Lab3.ipynb*, где Username — ваша фамилия на латинице, group — название группы (например, Struminskiy_IAD-20_Lab3.ipynb). Далее отправьте этот файл на используемую в Вашей группе почту курса (hse.minor.dm@gmail.com) c темой письма *[ИАД-NN] - Лабораторная работа 3 - Фамилия Имя Отчество*.

## Наивный байесовский классификатор

Обучение методом наивного Байеса основывается на достаточно сильном предположении, что все признаки независимы в совокупности. По формуле Байеса
$$P(y|x) = P(y|(x^1,\dots,x^d))=\frac{P(y)P(x^1,\dots,x^d|y)}{P(x^1,\dots,x^d)}.$$
В предположении, что признаки независимы, получаем, что 
$$P(y|x^1,\dots,x^d)=\frac{P(y)\prod_{i=1}^d P(x^i|y)}{P(x^1,\dots,x^d)}$$
Т.к. $P(x^1,\dots,x^d)$ не зависит от $y$, то формула наивного байесовского классификатора записывается следующим образом:

$$\hat y (x) = \arg\max_y P(y)\prod_{i=1}^d P(x^d|y).$$


Библиотека `sklearn` поддерживает несколько реализаций наивного Байеса: `sklearn.naive_bayes.GaussianNB`, `sklearn.naive_bayes.MultinomialNB` и др.

In [75]:
from sklearn.cross_validation import train_test_split
from sklearn.datasets import fetch_20newsgroups
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np

### Данные

Данными для задачи будет выступать подмножество одного из популярных датасетов [20 News Groups](http://qwone.com/~jason/20Newsgroups/), который является набором статей, разбитых по категориям (темам).

Пример статьи на тему космоса:
    
    From: MUNIZB%RWTMS2.decnet@rockwell.com ("RWTMS2::MUNIZB") 
    
    Subject: Alaska Pipeline and Space Station! 
    
    X-Added: Forwarded by Space Digest 
    
    Organization: [via International Space University] 
    
    Original-Sender: isu@VACATION.VENARI.CS.CMU.EDU 
    
    Distribution: sci 
    
    Lines: 16  
    
    on Date: 01 Apr 93 18:03:12 GMT, Ralph Buttigieg <ralph.buttigieg@f635.n713.z3.fido.zeta.org.au> writes: 
    
    Why can't the government just be a tennant? Private commercial concerns could just build a space station system and charge rent to the government financed researchers wanting to use it.  I believe that this was the thought behind the Industrial Space Facility.  I don't remember all the details, but I think Space Services (?) wanted NASA to sign an anchor tenancy deal in order to help secure some venture capital but NASA didn't like the deal.  (I'm sure I'll hear about it if I'm wrong!)  
    
    Disclaimer: Opinions stated are solely my own (unless I change my mind). 
    
    Ben Muniz     
    MUNIZB%RWTMS2.decnet@consrt.rockwell.com    w(818)586-3578 Space Station 
    Freedom:Rocketdyne/Rockwell:Structural Loads and Dynamics "Man will not fly for fifty years": Wilbur to Orville Wright, 1901

Можно заметить, что данные никак не предобработаны, пунктуация и служебная информация сохранены.


**Задание 1. (2 балла)** Выберите 2 произвольные категории текстов из списка ниже, с которыми вы будете работать в рамках данного задания. В переменной text находится список статей выборки, в переменной y — вектор ответов для них. Обратите внимание, что в качестве ответов выступают индексы тем в списке theme_list, а не сами названия тем. Выберите тексты, относящиеся к выбранным Вами темам (и сохраните сами тексты и вектор ответов для них в переменные text и y соответственно) и случайным образом, в пропорции 70/30, разделите наборы текстов на обучающую и контрольную выборки. В результате выполнения данного задания у вас должно получится 5 переменных: text, text_train, text_test, y_train, y_test.

Обратите внимание, что данные уже "перемешаны", поэтому достаточно просто поделить данные в пропорции 70/30.

In [76]:
# Список новостных заголовков
theme_list = 
['alt.atheism',
 'comp.graphics',     #1(потому что счет начинается с 0, а не с 1)
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',         #13
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

SyntaxError: invalid syntax (<ipython-input-76-c09c7f4d5d8d>, line 2)

In [77]:
dataset = fetch_20newsgroups(subset = 'train')
perm = np.random.permutation(pd.Series(dataset.data).index)
text_all = pd.Series(dataset.data).reindex(perm) #изменила названия, потому что на выходе в text и y должны лежать только выбранные тексты и ответы
y_all = pd.Series(dataset.target).reindex(perm)
text = []                                        #тексты по выбранным темам
y = []                                           #вектор ответов на них
for i in range(0, len(y_all) - 1):               #выбираем тексты
    if y_all[i] == 1 or y_all[i] == 13:
        text.append(text_all[i])
        y.append(y_all[i])
text_train, text_test, y_train, y_test = train_test_split(text, y, test_size=0.3, random_state=42) #делим на обучающую и контрольную

**Задание 2. (2 балла)** На данный момент данные имеют текстовый вид, и не очень понятно, как с ними работать дальше. В этом задании мы приведем их к числовому представлению. Для каждого текста признаками будут являться количество вхождений конкретного слова в данный текст. Например, для выборки из двух текстов {"one two two", "one three two"} матрицей "объект-признак" будет являться

$$\left[ \begin{matrix} 1 & 2 & 0 \\ 1 & 1 & 1 \end{matrix} \right],$$

где номер строки в матрице совпадает с номером текста в выборке, признаки обозначают количество вхождений слова "one", "two" или "three" соответственно (слева направо).

[CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) — объект, позволяющий привести выборку, состоящую из текстов, к указанному выше виду. Необходимый объект уже создан в ячейке ниже.
* Метод **fit** объекта CountVеctorizer позволяет "сохранить" в его внутренней структуре набор всех уникальных слов из передаваемого параметра.
* Метод **transform** этого же объекта позволяет привести набор текстов, переданный в качестве параметра, к указанному выше виду.

Вызовите метод fit объекта vectorizer из ячейки ниже, передав в качестве параметра набор текстов text, — таким образом он "запомнит" все имеющиеся в выборке слова. Затем дважды вызовите метод transform, передав ему параметры text_train и text_test соответственно — таким образом вы получите необходимое представление ваших обучающей и контрольной выборок. Сохраните их в переменные X_train и X_test соответственно — таким образом, вы получили 2 матрицы "объекта-признак". Обратите внимание, что эти матрицы являются сильно разреженными, поэтому вызовите для каждой из них метод .toarray(), чтобы привести их в типу np.ndarray.

In [78]:
vectorizer = CountVectorizer(analyzer='word', stop_words='english', lowercase=True)
wordlist = vectorizer.fit(text)                       #список всех уникальных слов в выбранных текстах
X_train = vectorizer.transform(text_train).toarray()  #создаем матрицу "объект-признак" и приводим к типу np.ndarray обучающую выборку
X_test = vectorizer.transform(text_test).toarray()    #создаем матрицу "объект-признак" и приводим к типу np.ndarray контрольную выборку 

**Задание 3. (3 балла)** По матрице X_train выведите 50 наиболее часто встречающихся слов.

Для этого вам сначала необходимо сложить все строки X_train, чтобы найти общее количество вхождений для каждого слова. Далее используйте метод argsort() (выполняет сортировку вектора по возрастанию, однако возвращает вектор не значений, а их **индексов**). Таким образом, 50 последних значений в получившемся векторе — это номера признаков, которые отвечают за самые частые слова. Теперь, чтобы получить сами слова, используйте метод vectorizer.get_feature_names() (возвращает список слов в том порядке, в котором расположены отвечающие им столбцы в матрице X_train).

In [79]:
X_train_freq = np.argsort(np.sum(X_train, axis=0))[-50:]   #суммируем X_train по строкам, сортируем по возрастанию и берем 50 последних значений
words = vectorizer.get_feature_names()                     #список всех слов
for i in X_train_freq:                                     #наиболее часто встречающиеся слова
    print (words[i])

gordon
version
thanks
10
data
ftp
need
ve
research
files
reply
medical
think
software
health
pitt
mail
good
available
help
science
program
new
information
jpeg
people
computer
ca
file
time
does
just
nntp
host
image
like
cs
know
posting
don
use
graphics
writes
university
article
organization
lines
subject
com
edu


**Задание 4. (2 балла)** Обучите наивный байесовский классификатор MultinomialNB с параметром alpha=1 на выборке train. Примените его к данным контрольной выборки test и выведите долю верно классифицированных текстов.

[Пример](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html) использования MultinomialNB (раздел Examples).
Данный объект имеет 2 необходимых вам метода:
* fit(X_train, y_train) — обучение наивного байесовского классификатора
* predict(X_test) — предсказание ответов обученного классификатора на контрольной выборке

In [80]:
clf = MultinomialNB(alpha = 1)
clf = clf.fit(X_train, y_train)                                   #обучаем классификатор
predict_y = clf.predict(X_test)                                   #предсказываем значения
correct = np.array([y_test == predict_y])                         #создаем массив, попарно сравнивая значения контрольных у-ков и предсказанных
correct = correct.flatten()                                       #убираем лишние []
percentage = (float(np.sum(correct)) / float(len(correct))) * 100 #превращаем True в 0, а False в 1, складываем - получаем сумму всех 
print (percentage, '%')                                           #верно предсказанных у, делим ее на количество всех у

99.15254237288136 %


**Задание 5. (1 балл)** Обучите наивный байесовский классификатор MultinomialNB с параметром alpha=0 на выборке train. Примените его к данным контрольной выборки test и выведите долю верно классифицированных текстов. Поменяются ли результаты?

In [81]:
clf = MultinomialNB(alpha = 0)          #то же самое, только альфа = 0
clf = clf.fit(X_train, y_train)                                   
predict_y = clf.predict(X_test)                                   
correct = np.array([y_test == predict_y])                         
correct = correct.flatten()                                       
percentage = (float(np.sum(correct)) / float(len(correct))) * 100   
print (percentage, '%') 

51.41242937853108 %


Результат поменялся. С альфа = 0 результат почти в 2 раза хуже